In [77]:
from bs4 import BeautifulSoup
import requests
import csv
import re
from requests_html import HTMLSession # https://github.com/psf/requests-html

On cherche par rapport au nom de famille, les noms de famille de naissance sont utilisés. Il faut donc regarder le lieu de naissance des enfants pour savoir si la famille à bouger (suivie le/la partenaire).

Il faudrait arriver à trouver l'ancêtre le plus loin pour un nom de famille, puis analyser sa descendance :
Pour chaque enfant : 
    - Si c'est un garcon ou une fille on récupère les informations nescessaires + récupération des informations de l'épouse/époux
    - Si c'est un garcon on analysera la descendance


Informations à récupérer pour chaque personne :
 - Genre
 - Nom
 - Prénom
 - Date naissance
 - Lieu de naissance
 - Date mort
 - Lieu de mort


Visualisation (folium):
 - Sur une carte qui s'actualise tous les 10 ans, on affiche les naissances et les morts aux endroits correspondants

Avec le lieu de naissance de toute la famille, on peut voir comment ils se sont déplacé au fil des ans.

Améliorations :

Utilisation du lieu de mort, du lieu de certains événements, des informations des époux/épouses




In [78]:
# set up 
# !pip install selenium
# !pip install bs4
# !pip install webdriver_manager
# !pip install requests


In [79]:
# initialisation

# vidéo tuto : https://www.youtube.com/watch?v=3fcKKZMFbyA 


data_test = [{"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"},{"nom":"MARTIN", "prénom":"Jacques", "genre": "M", "ddn": 1822, "ldn":"Lyon", "ddm": 1898, "ldm":"Paris"}]

# print(data_test)
data_test = sorted(data_test, key=lambda d: d['ddn']) 
# print(data_test)


In [80]:
# Futur fonction à utiliser (pour saisie du nom prenom et création de l'url correspondante)
def traitementURL():
    nom, prenom=getNomPrenom()
    url = "https://www.geneanet.org/fonds/individus/?go=1&nom=" + nom + "&prenom=" + prenom + "&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10" 
    return url 

def getNomPrenom():
    nom = input("Saisir nom")
    prenom = input("Saisir prénom")
    return nom, prenom

def getAncetres():
    nom, prenom = getNomPrenom()
    # récupérer f&s

    # récupérer père et mère 
    
    return

In [81]:
# Url de recherche de test
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&prenom=fran%C3%A7ois&prenom_operateur=or&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint=&size=10"

In [82]:
# Récupération du code html
def getSoup(url):
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup

In [83]:
url = "https://www.geneanet.org/fonds/individus/?go=1&nom=martin&page=1&prenom=fran%C3%A7ois&prenom_operateur=or&size=10&with_variantes_nom=&with_variantes_nom_conjoint=&with_variantes_prenom=&with_variantes_prenom_conjoint="

# Récupération du code html
soup = getSoup(url)

# Récupération de la div contenant toutes les urls en lien avec la recherche
resOnly = soup.find("div", {"id":"table-resultats"})
# print(resOnly)

allArbreUrl = []
# Récupération des urls
for a in resOnly.find_all('a'):
    try:
        if (re.search("^arbres_utilisateur", a["data-id-es"])):  #^arbres_utilisateur
            allArbreUrl.append(a["href"])
    except:
        ""
print(allArbreUrl) #Comporte l'url des pages utilisateurs si se sont des arbres utilisateurs
# headline = a.text-large
# print(headline)

['https://gw.geneanet.org/marmi1?n=martin&oc=&p=francois', 'https://gw.geneanet.org/martin86?n=martin&oc=&p=francois', 'https://gw.geneanet.org/gdemar?n=martin&oc=&p=francois', 'https://gw.geneanet.org/mduvernoy?n=martin&oc=3&p=francois', 'https://gw.geneanet.org/erikobay?n=martin&oc=&p=francois+jean+francois+claude', 'https://gw.geneanet.org/jgerme?n=martin&oc=38&p=francois', 'https://gw.geneanet.org/moixou?n=martin&oc=&p=francois', 'https://gw.geneanet.org/cms71?n=martin&oc=8&p=francois', 'https://gw.geneanet.org/chacha232?n=martin&oc=10&p=francois', 'https://gw.geneanet.org/fbourgarel?n=martin&oc=2&p=francois']


In [84]:
session = HTMLSession()

In [85]:
# Récupération des ascendants (remonter facilement à l'ancetre)
def getUrlDerAscendant(url):
    print(url)
    r = session.get(url)
    soup = BeautifulSoup(r.html.html, 'html.parser')
    # Récupération du dernier ascendants
    # print(soup)
    urlDerAscendant = url
    try: 
        arbreGene =  soup.find("td", {"id":"ancestors"})
        derAscendant = arbreGene.find("a")["href"]
        urlDerAscendant = "https://gw.geneanet.org/" + derAscendant
        urlDerAscendant = getUrlDerAscendant(urlDerAscendant)
        
    except:
        print("Pas d'autre ascendent")

    return urlDerAscendant

In [86]:
def getEpouxEnfant(person_html):
    # Récupération époux/épouse/enfants
    person_union = person_html.find("ul", {"class":"fiche_union"})
    person_epoux = person_union.findChildren("li" , recursive=False)
    urlEpoux = []
    urlChildren = []
    data = []
    try:
        for epoux in person_epoux:
            urlEpoux.append("https://gw.geneanet.org/" + epoux.find("a")["href"])
            children  = epoux.find_all("li")
            try:
                for child in children:
                    urlChildren.append("https://gw.geneanet.org/" + child.find("a")["href"])
            except:
                print("Err Children")
    except:
        print("Err Epoux")
    
    for url in urlEpoux:
        data += getDataUrl(url)
    for url in urlChildren:
        data += getDataUrl(url)

    return data

In [87]:
def getDataUrl(url):
    data = []
    personne = {}
    print(url)
    r = session.get(url)
    soup = BeautifulSoup(r.html.html, 'html.parser')

    
    # Récupération de l'html de l'ensemble des données d'une personne
    person_html = soup.find("div", {"id":"perso"})

    # Récupération des données concernant la personne (genre, nom, prenom)
    person_title = person_html.find("div", {"id":"person-title"})
    person_title = person_title.find("h1")

    personne["genre"] = person_title.find("img")["title"]
    
    nomprenom = person_title.find_all("a")
    personne["nom"] = nomprenom[0].text
    personne["prenom"] = nomprenom[1].text

    # Récupération des données concernant la personne (ddm, ldm, ddm, ldm)
    person_data_perso = person_html.find("ul")

    for li in person_data_perso.findAll("li"):

        try:
            print(li.text)
            print(re.findall("[0-9]{4}", re.findall("Né.*", li.text)[0])[0])
            personne["ddn"] = re.findall("[0-9]{4}", re.findall("Né.*", li.text)[0])[0]
            
        except:
            ""
        try:
            print(li.text)
            print(re.findall("[0-9]{4}", re.findall("Décédé.*", li.text)[0])[0])
            personne["ddm"] = re.findall("[0-9]{4}", re.findall("Décédé.*", li.text)[0])[0]
        except:
            ""
    
    # date_naissance = person_data_perso.find("li").text
    # print(date_naissance)
    # print(re.search("Né le .* -", date_naissance))

    personne["ldn"] = ""
    personne["ldm"] = ""
    
    
    # Récupération des parents
    try:
        parents_data = person_html.find("div", {"id":"parents"})
        personne["pere"] = parents_data.findAll("a")[0].text
        personne["mere"] = parents_data.findAll("a")[1].text
    except:
        print("Erreur parents")

    print(personne)


    
    if(personne["genre"] == "H"):
        try:
            data = getEpouxEnfant(person_html)
        except:
            print("Err Epoux/Enfant")
    data.append(personne)
    print(data)
    return data

    

In [88]:

session = HTMLSession()
url = "https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=pierre&n=martin&oc=12"

data = []
personne = {}
print(url)
r = session.get(url)
soup = BeautifulSoup(r.html.html, 'html.parser')


# Récupération de l'html de l'ensemble des données d'une personne
person_html = soup.find("div", {"id":"perso"})

# Récupération des données concernant la personne (genre, nom, prenom)
person_title = person_html.find("div", {"id":"person-title"})
person_title = person_title.find("h1")

personne["genre"] = person_title.find("img")["title"]

nomprenom = person_title.find_all("a")
personne["nom"] = nomprenom[0].text
personne["prenom"] = nomprenom[1].text

# Récupération des données concernant la personne (ddm, ldm, ddm, ldm)
person_data_perso = person_html.find("ul")

for li in person_data_perso.findAll("li"):

    try:
        print(li.text)
        print(re.findall("[0-9]{4}", re.findall("Né.*", li.text)[0])[0])
        personne["ddn"] = re.findall("[0-9]{4}", re.findall("Né.*", li.text)[0])[0]
        
    except:
        ""
    try:
        print(li.text)
        print(re.findall("[0-9]{4}", re.findall("Décédé.*", li.text)[0])[0])
        personne["ddm"] = re.findall("[0-9]{4}", re.findall("Décédé.*", li.text)[0])[0]
    except:
        ""

# date_naissance = person_data_perso.find("li").text
# print(date_naissance)
# print(re.search("Né le .* -", date_naissance))

personne["ldn"] = ""
personne["ldm"] = ""


# Récupération des parents
try:
    parents_data = person_html.find("div", {"id":"parents"})
    personne["pere"] = parents_data.findAll("a")[0].text
    personne["mere"] = parents_data.findAll("a")[1].text
except:
    print("Erreur parents")

print(personne)

https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=pierre&n=martin&oc=12
Né vers 1668 - peut être à Chissey en Morvan
1668
Né vers 1668 - peut être à Chissey en Morvan
Décédé le 11 juin 1740 - à Lucenay l'Evèque,
à l'âge d'environ 72 ans
Décédé le 11 juin 1740 - à Lucenay l'Evèque,
à l'âge d'environ 72 ans
1740
Inhumé le 12 juin 1740 - à Lucenay l'Evèque
Inhumé le 12 juin 1740 - à Lucenay l'Evèque
Cabartier à Souvert parroise alternative de Chissey et Lucenay
Cabartier à Souvert parroise alternative de Chissey et Lucenay
{'genre': 'H', 'nom': 'Pierre', 'prenom': 'Martin', 'ddn': '1668', 'ddm': '1740', 'ldn': '', 'ldm': '', 'pere': 'Léger Martin', 'mere': 'Etiennette Bourguinier'}


In [89]:
test = """Décédé le 11 juin 1740 - à Lucenay l'Evèque,
à l'âge d'environ 72 anséeée"""
print(re.findall("[0-9]{4}", re.findall("Décédé.*", test)[0])[0])

1740


In [90]:
# Récupération du code source de chaque url correspondant à une personne
# {"nom":"MARTIN", "prénom":"Jean", "genre": "M", "ddn": 1855, "ldn":"Paris", "ddm": 1920, "ldm":"Paris"}
urlDerAscendant = ""
# for url in allArbreUrl:
for i in range(1):
    url = "https://gw.geneanet.org/marmi1?n=martin&oc=&p=francois"
    print("url de base : " + url)
    data = []
    
    urlDerAscendant = getUrlDerAscendant(url)
    print("url dernier ascendant : " + urlDerAscendant)
    data += getDataUrl(urlDerAscendant)
    print(data)


url de base : https://gw.geneanet.org/marmi1?n=martin&oc=&p=francois
https://gw.geneanet.org/marmi1?n=martin&oc=&p=francois
https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=noel&n=martin
https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=jean&n=martin&oc=18
https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=mathieu&n=martin
Pas d'autre ascendent
url dernier ascendant : https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=mathieu&n=martin
https://gw.geneanet.org/marmi1?lang=fr&pz=mathieu&nz=martin&p=mathieu&n=martin
Né peut-être en 1590 - ( Souvert) commune alternative de Chissey en Morvan et Lucenay l'Evèque en Bourgogne à la fin du 16eme siècle.
1590
Né peut-être en 1590 - ( Souvert) commune alternative de Chissey en Morvan et Lucenay l'Evèque en Bourgogne à la fin du 16eme siècle.
Décédé
Décédé
Erreur parents
{'genre': 'H', 'nom': 'Mathieu', 'prenom': 'Martin', 'ddn': '1590', 'ldn': '', 'ldm': ''}
https://gw.geneanet.org/marmi1?lang=fr&pz=m